In [1]:
! pip install transformers
# ! pip install transformers<4.38
# ! pip install tensorflow==2.11.0 keras==2.11.0

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
! pip install requests zipfile


ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [5]:
import requests
import zipfile
import os

# zip 파일 URL
zip_url = "https://github.com/seoyeokim/Aiffel_DLThon/raw/main/data/preprocessed_train_raw.zip"

# zip 파일 다운로드
response = requests.get(zip_url)
zip_path = "preprocessed_train_raw.zip"

# 다운로드한 zip 파일 저장
with open(zip_path, "wb") as file:
    file.write(response.content)

# 압축 해제
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("preprocessed_train_raw")

print("압축 해제 완료!")

압축 해제 완료!


In [6]:
import os
import pandas as pd

# 압축 해제된 폴더 경로
folder_path = "preprocessed_train_raw"

# 폴더 내의 모든 파일을 확인
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 첫 번째 CSV 파일을 데이터프레임으로 읽기
csv_path = os.path.join(folder_path, csv_files[0])  # 첫 번째 CSV 파일 경로
train_data = pd.read_csv(csv_path)

# 데이터프레임 출력
train_data.head()

,Unnamed: 0,conversation,class
0,0,아이가 아빠 생일이라고 그림을 그려줬어.\n좋으시겠어요. 자녀분이 그림을 잘 그리나...,4
1,1,야 안녕\n .어 안녕.\n 나 오늘 택시비가 없는데 혹시 빌려줄 수 있어 ??\n...,1
2,2,친구들이 내 결혼식장에 오면 우리 부부를 보여주는 게 용기가 나질 않아.\n친구들에...,4
3,3,아들이 이번에 고등학교 졸업했어.\n이번에 졸업을 하셨다니 졸업 축하 드려요.\n근...,4
4,4,나를 괴롭히던 친구들이 전학을 갔고 안 좋은 사건들을 선생님이 해결해 주어서 기뻐....,4


In [30]:
print(len(train_data))

67640


In [9]:
test = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/test.csv"
test_data = pd.read_csv(test)
test_data.head()

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [10]:
# #전처리한 test data

# def data_import(data_name):
#     data_path = os.getenv('HOME')+'/aiffel/project_data/DLThon/'+data_name
#     imported_data = pd.read_csv(data_path)
#     return imported_data

# new_test = data_import('new_test.csv')

# new_test.head()

,idx,text
0,t_000,아가 씨 담 배 한 갑 주소 네\n4500 원 입니다 어 네\n지 갑 어 디 갔...
1,t_001,우리 팀 에서 다른 팀 으로 갈 사람 없 나 ? 그럼 영 지 씨가 가는 건 어 때...
2,t_002,너 오늘 그 게 뭐 야 네\n제가 뭘\n잘못 했\n나요 . ? 제대로 좀\n하지...
3,t_003,이 거 들어 바 와 이\n노래 진짜 좋다\n그\n치\n요즘 이 것 만 들어 진짜 ...
4,t_004,아무 튼 앞으로 니 가 내 와이 파이 야 . \n. 응 와이\n파이 온 . 켰 ...


In [13]:
train_data['conversation'][0]

'아이가 아빠 생일이라고 그림을 그려줬어.\n좋으시겠어요. 자녀분이 그림을 잘 그리나 봐요?\n그런 거 같아. 취미로 시작한 건데 본인이 좋아하고 또 적성에 맞나 보네.\n그러면 계속 그림 쪽으로 진로를 계획하고 계시나요?\n본인이 원하면 계속 도와주려고 하고 있어.\n아이의 진로를 응원해 주는 좋은 부모님이신 거 같네요.'

In [39]:
train_texts = train_data["conversation"].to_list() # Features (not-tokenized yet)
train_labels = train_data["class"].to_list() # Labels

In [40]:
# validation data 분리

from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=0)

### 토크나이저 다운로드

In [17]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# 시작 토큰, 종료 토큰 붙이기

import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences

def convert_examples_to_features(sent_list, max_seq_len, tokenizer):
    input_ids = []

    for conversation in tqdm(sent_list, total=len(sent_list)):
        bos_token = [tokenizer.bos_token]  # 시작 토큰
        eos_token = [tokenizer.eos_token]  # 종료 토큰
        unused_token = ['<unused0>']  # 마지막에 추가할 토큰

        # '\n'으로 문장들 분리
        sentences = conversation.split("\n")

        tokens = []  # 전체 토큰 리스트

        for i, sentence in enumerate(sentences):
            # 각 문장에 <s>와 </s> 추가
            sentence_tokens = bos_token + tokenizer.tokenize(sentence) + eos_token
            tokens += sentence_tokens  # 문장 뒤에 <s> 추가

        # 마지막 문장 뒤에 <unused0> 추가
        tokens += unused_token  # 마지막에 <unused0> 추가
        tokens += eos_token  # 마지막에 종료 토큰 추가

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        # 패딩 추가
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        # 길이 확인
        assert len(input_id) == max_seq_len, f"Error with input length {len(input_id)} vs {max_seq_len}"

        # 결과 저장
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids


In [20]:
# maxlen 계산하기

total_data_text = list(train_texts)
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))
 
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

문장길이 평균 :  188.1513712300414
문장길이 최대 :  1091
문장길이 표준편차 :  53.84390376602451
pad_sequences maxlen :  295
전체 문장의 0.9754398285038439%가 maxlen 설정값 이내에 포함됩니다. 


In [21]:
max_seq_len = 300

# 학습데이터 전처리
X_train = convert_examples_to_features(train_texts, max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 54112/54112 [00:36<00:00, 1488.83it/s]


In [25]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    0  9245  8344 12961 24692  9106  7239 18954 12396 14493  9081 15053
 12583  8006   389  9443 12680  7080  6889 11190  7406  8263 13656  9094
  9183  9897 27515  7652  9668 16172 14337  8022  9327  9711 23044 10065
  8017  8006   389     1     0  9245  8344 12961  9106  7239 18954 12396
 14493 12474 20486  9081 15053  9586  9122  6921 25856     1     0  9769
 15076 19319  9022 12961  7426 46439 25990  8815  8137 38492  9868  7549
  8137 11093 31204   389  9252  7220  9022  9728 11519 31665 22601 10125
  7372  7788 11190  7406  8263 13656 45023  9183  9897 27515 12399  9539
 47804     1     0 15076  9868  7549  8022 18655  7481 11841 21466 15148
 11219  7816  6921 25856     1     0 17582  9022 12396  9134 46651  9329
 35482 12442 34019  6969   389 10099  9355  9063 15076 20164  9217 12980
 34261 19935 13289 18050  9166  7894 10452  9024 10178 13493 12704  9934
  9193  8263 42138  8006   389     1     0  9166 13922 21831  9024 10178
  9076  8702 11768  9031 27511 2585

In [26]:
# 검증데이터 전처리

X_val = convert_examples_to_features(val_texts, max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 13528/13528 [00:09<00:00, 1470.53it/s]


In [28]:
# GPT 모델 불러오기

model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [29]:
# GPT 모델 input, output 만들기
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [31]:
# GPT로 다중 분류 모델 만들기

class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [32]:
# # TPU 작동을 위한 코드
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [33]:
# 모델 생성
from tensorflow.keras.metrics import Recall

model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=5)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# recall_metric = Recall()
# model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', recall_metric])
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [47]:
import numpy as np

# train_labels와 val_labels가 리스트라면, 이를 NumPy 배열로 변환
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [ ]:
model.fit(
    X_train, train_labels, epochs=1, batch_size=16, validation_data = (X_val, val_labels)
)

1084/3382 [========>.....................] - ETA: 42:41 - loss: 0.1071 - accuracy: 0.9673

In [ ]:
# test_data 전처리 (\n으로 구분하지 않고 대화문 통으로 하나의 문장으로 설정)

def preprocess_test_data_without_newline(test_data, tokenizer, max_seq_len=128):
    # test_data에서 conversation만 추출
    test_texts = test_data["text"].to_list()  # Features (not-tokenized yet)

    # 전처리 진행
    def convert_examples_to_features_for_test(sent_list, max_seq_len, tokenizer):
        input_ids = []

        for conversation in tqdm(sent_list, total=len(sent_list)):
            bos_token = [tokenizer.bos_token]  # 시작 토큰
            eos_token = [tokenizer.eos_token]  # 종료 토큰
            unused_token = ['<unused0>']  # 마지막에 추가할 토큰

            # conversation을 하나의 문장으로 간주
            sentence = conversation.strip()

            tokens = bos_token + tokenizer.tokenize(sentence) + eos_token  # 토큰화 후 <s>, </s> 추가

            # 마지막에 <unused0> 추가
            tokens += unused_token  # 마지막에 <unused0> 추가
            tokens += eos_token  # 마지막에 종료 토큰 추가

            # 정수 인코딩
            input_id = tokenizer.convert_tokens_to_ids(tokens)
            # 패딩 추가
            input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

            # 길이 확인
            assert len(input_id) == max_seq_len, f"Error with input length {len(input_id)} vs {max_seq_len}"

            # 결과 저장
            input_ids.append(input_id)

        input_ids = np.array(input_ids, dtype=int)

        return input_ids

    # test_data에 대해 전처리 진행
    X_test = convert_examples_to_features_for_test(test_texts, max_seq_len, tokenizer)

    return X_test

# 테스트 데이터 전처리 실행
X_test = preprocess_test_data_without_newline(test_data, tokenizer, max_seq_len=128)

print("Test data preprocessing 완료!")


In [ ]:
# 레이블과 정수 매핑 딕셔너리
label_mapping = {0: 1, 1: 3, 2: 4, 3: 2, 4: 0}           # 랜덤 정수 라벨링은 캐글 기준으로 바꾸기

# 예측 수행
predictions = model.predict(X_test)

# 예측 결과가 클래스 확률일 경우
predicted_classes = np.argmax(predictions, axis=1)

# 정수 값을 레이블로 변환
predicted_labels = [label_mapping[class_id] for class_id in predicted_classes]

# 예측 결과를 test 데이터프레임에 추가
test_data['predictions'] = predicted_classes
test_data.to_csv("test.csv",index=False, encoding='utf-8-sig')


# 결과 출력
test_data.head()

In [ ]:
test_data.sample(10)

생성 데이터로 다시 해보면 좋을 듯 하다

In [ ]:
# submission 넣기

submission = pd.DataFrame({"idx" : test_data.idx, "class" : predicted_labels})
submission.to_csv("submission.csv",index=False, encoding='utf-8-sig')

submission.head()